In [ ]:
import pandas as pd
truck_age_data = pd.read_excel('/content/Age of Trucks.xlsx')

In [ ]:
truck_age_data['Age of Truck'].values

array([ 5,  3,  2,  1,  4,  2, 10,  7,  5,  5,  3,  8,  4,  8, 10,  4,  8,
        1,  9,  8,  2,  1,  8,  3,  1, 10,  7,  2,  6,  8,  4, 10,  9,  1,
        2,  2,  5, 10,  3,  3,  9,  1,  9,  7,  4,  4,  9,  9,  5,  5,  7,
        5,  3,  7,  3,  2,  9,  4,  8,  8, 10,  6,  5,  5,  4,  7,  4,  2,
        5,  5,  2,  6,  2,  1, 10,  3,  2,  5,  2,  2])

In [ ]:
population_size = 500*80 #the number of simultaneous solution that can exist together.
generations = 100 # total number of iterations for getting schedules
mutation_rate = 0.4
elitism_rate = 0.25 # choosing 25 best solns to build upon them

TO HAVE A DYNAMIC APPROACH, WE CAN CHANGE THE PARAMETERS LIKE TRUCKS AVAILABLE AT THE MOMENT, PRODUCT TYPES ,ETC

In [ ]:
import numpy as np
import pandas as pd
import random

TRUCKS = 5
TRUCK_TYPES = ["Perishable", "Fragile", "Electronics", "General"]
DAYS = 5
MIN_HOURS = {"Perishable": 3000, "Fragile": 4000, "Electronics": 5000, "General": 6000}
MAX_HOURS = {"Perishable": 4000, "Fragile": 6000, "Electronics": 8000, "General": np.inf}
MAX_CONTINUOUS_DAYS = 3
MAX_DAILY_HOURS = 16
MONTHLY_HOUR_REQUIREMENT = 27000
WEEKLY_HOUR_REQUIREMENT = 50

# Additional constants for revenue
REVENUE_TIERS = [0, 200, 250, 300, 400] #hours travelled category
REVENUE_RATES = [500, 550, 600, 700, 800] #rate per hour for truck
ADDITIONAL_REVENUE_TYPES = {"Perishable": 0.10, "Fragile": 0.08, "Electronics": 0.05, "General": 0.0}
ADDITIONAL_REVENUE_AGES = {"0-2": 0.10, "2-5": 0.05, "5+": 0.0}

# Genetic Algorithm parameters

## Assuming we have 500 delivery orders/day total population siez we can get is x-x (500x80)


population_size = 10  #the number of simultaneous solution that can exist together. IF WE HAVE 30 DELIVERIES TODAY WITH 40 TRUCKS AVAILABLE SO 40*30, MAKE CHANGES TO THE TRUCK_TYPES LIST ALSO
generations = 5 # total number of iterations for getting schedules
mutation_rate = 0.4
elitism_rate = 0.25 # choosing 25 best solns to build upon them

# Initialization of the population
population = np.random.rand(population_size, TRUCKS * len(TRUCK_TYPES) * DAYS)


truck_age_data = pd.read_excel('/content/Age of Trucks.xlsx')
age_values=truck_age_data['Age of Truck'].values

def calculate_fitness(solution, age_data):
    schedule = solution.reshape((TRUCKS, len(TRUCK_TYPES), DAYS))
    total_revenue = 0

    for i in range(TRUCKS):
        for k in range(DAYS):
            for j, rate in enumerate(REVENUE_RATES[:len(TRUCK_TYPES)]):
                hours_value = schedule[i, j, k]

                age_category = "5+"  if k>5 else "other"
                travel_hours=random.randint(0,1000)
                if(travel_hours<=200):
                   base_revenue=500*travel_hours
                elif(200<travel_hours<=250):
                  base_revenue=550*travel_hours
                elif(250<travel_hours<=300):
                  base_revenue=600*travel_hours
                elif(300<travel_hours<=400):
                  base_revenue=700*travel_hours
                else:
                  base_revenue=800*travel_hours
                additional_revenue = (
                    ADDITIONAL_REVENUE_TYPES[TRUCK_TYPES[j]] +
                    ADDITIONAL_REVENUE_AGES["0-2"] * (k <= 2) +
                    ADDITIONAL_REVENUE_AGES["2-5"] * (2 < k <= 5)
                )*base_revenue

                daily_revenue = base_revenue+ additional_revenue * hours_value * rate # missed adding base revenue

                print(f"Truck ID: {i}, Day: {k + 1}, Type: {TRUCK_TYPES[j]}, Hours: {hours_value}, Age Category: {age_category}, Additional Revenue: {additional_revenue}, Daily Revenue: {daily_revenue}")

                total_revenue += daily_revenue  # Accumulate daily revenue to total revenue

    return total_revenue

# Genetic Algorithm
for generation in range(generations):
    # Calculate fitness for each solution in the population

    fitness_values = np.array([calculate_fitness(solution, age_values) for solution in population])

    # Select the top-performing solutions (parents)
    selected_indices = np.argsort(fitness_values)[-int(population_size * elitism_rate):]
    parents = population[selected_indices]

    # Generating new solutions through crossover SBX technique
    offspring = np.zeros_like(population)
    for i in range(len(parents), population_size):
        parent_indices = np.random.choice(len(parents), size=2, replace=False)
        parent1, parent2 = parents[parent_indices[0]], parents[parent_indices[1]]

        # Simulated Binary Crossover (SBX)
        beta = np.random.uniform(0, 1, parent1.shape)
        beta = np.where(beta <= 0.5, (2 * beta) ** (1 / (1 + 1)), (1 / (2 * (1 - beta))) ** (1 / (1 + 1)))
        offspring[i] = 0.5 * ((1 + beta) * parent1 + (1 - beta) * parent2)

        adaptive_mutation_rate = mutation_rate / (generation + 1) if generation > 0 else mutation_rate
        mutation_mask = np.random.rand(*offspring.shape) < adaptive_mutation_rate
        offspring += mutation_mask * np.random.rand(*offspring.shape)

    # Combining both parents and offspring to form the next generation
    population[:len(parents)] = parents
    population[len(parents):] = offspring[:population_size - len(parents)]

# Select the maximum from the final generation
best_solution = population[np.argmax([calculate_fitness(solution, age_values) for solution in population])]

# Reshape the best solution
best_schedule = best_solution.reshape((TRUCKS, len(TRUCK_TYPES), DAYS))

results_df = pd.DataFrame(columns=["Truck", "Day", "Type", "Hours", "DailyRevenue"])
for i in range(TRUCKS):
    for k in range(DAYS):
        for j in range(len(TRUCK_TYPES)):
            hours_value = best_schedule[i, j, k]
            if hours_value > 0:
                age_category = "5+" if k > 5 else "Other"
                additional_revenue = (
                    ADDITIONAL_REVENUE_TYPES[TRUCK_TYPES[j]] +
                    ADDITIONAL_REVENUE_AGES["0-2"] * (k <= 2) +
                    ADDITIONAL_REVENUE_AGES["2-5"] * (2 < k <= 5)
                )
                daily_revenue = additional_revenue * hours_value * REVENUE_RATES[j]
                new_df=pd.DataFrame({
                    "Truck": [i],
                    "Day": [k + 1],
                    "Type": [TRUCK_TYPES[j]],
                    "Hours": [hours_value],
                    "DailyRevenue": [daily_revenue]
                })
                results_df = pd.concat([results_df,new_df], ignore_index=True)

results_df.to_csv("optimal_schedule.csv", index=False)

Streaming output truncated to the last 5000 lines.
Truck ID: 0, Day: 1, Type: Perishable, Hours: 0.7126673700527615, Age Category: other, Additional Revenue: 78720.0, Daily Revenue: 28444187.68527669
Truck ID: 0, Day: 1, Type: Fragile, Hours: 0.6093779966118046, Age Category: other, Additional Revenue: 92448.0, Daily Revenue: 31498277.36692246
Truck ID: 0, Day: 1, Type: Electronics, Hours: 0.5562864706524492, Age Category: other, Additional Revenue: 120000.00000000001, Daily Revenue: 40852625.88697635
Truck ID: 0, Day: 1, Type: General, Hours: 0.7054073255330594, Age Category: other, Additional Revenue: 54880.0, Daily Revenue: 27647727.81767801
Truck ID: 0, Day: 2, Type: Perishable, Hours: 0.9177384789581008, Age Category: other, Additional Revenue: 148640.0, Daily Revenue: 68949523.75616606
Truck ID: 0, Day: 2, Type: Fragile, Hours: 0.853806161240328, Age Category: other, Additional Revenue: 74304.0, Daily Revenue: 35305467.15264074
Truck ID: 0, Day: 2, Type: Electronics, Hours: 0.935

In [ ]:
results_df